### Suposiciones
- La red es fija y la tabla se llama red
- La tabla de vértices se llama red_vertices_pgr
- tiene una columna que se llama costo
- tiene campos source y target
- todas las columnas de geometría se llaman geom

Primero todos los imports

In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from functools import reduce
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin
import io
from subprocess import call


### Parámetros de la conexión
Aquí servidor, usuario, password y base de datos están fijos, en realidad hay que resolver cómo funciona esto en la aplicación

In [2]:
db = 'cadenas'
usr = 'postgres' 
psw = 'postgres' 
host = '192.168.18.22'

In [3]:
con = psycopg2.connect(database= db, user=usr, password=psw, host=host)
engine = create_engine('postgresql://{}:{}@{}:5432/{}'.format(usr,psw,host,db))

### Etapas de la cadena
Variable con los nombre de las tablas de etapas. Esta tiene que venir de la interfase

In [4]:
etapas = ['etapa1', 'etapa2', 'etapa3', 'etapa4' ]

### Relaciones entre nodos y etapas

Asignar a cada punto de cada etapa el nodo más cercano de la red. Se crean las tablas con las relaciones:`etapa_i_node`, cada tabla tiene dos columnas, el id de la tabla de etapas y el id de la tabla de nodos.


In [5]:
def node_relations(table, engine, connection):
    """Toma una tabla de puntos y le agrega una columna con el id del
       nodo más cercano de la red.
       Elimina la tabla de relaciones en caso de que exista.
       Regresa el dataframe con la relación.
    """
    drop_qry = """drop table if exists %(tabla)s_node""" % {"tabla": table}
    curs = connection.cursor()
    curs.execute(drop_qry)
    connection.commit()
    sql = """
            select f.id as id_%(tabla)s, (
              SELECT n.id
              FROM red_vertices_pgr As n
              ORDER BY f.geom <-> n.geom LIMIT 1
            )as closest_node
            from %(tabla)s f
          """ % {"tabla": table}
    try:
        df = pd.read_sql(sql, engine)
    except ValueError as e:
        print(e)
    try:
        df.to_sql(table + '_node', engine)
    except ValueError as e:
        print(e)
    return df

Entonces, para agregar todas las tablas de relaciones hacemos:

In [42]:
node_relations_list = []
for etapa in etapas:
    node_relations_list.append(node_relations(etapa, engine, con))

In [9]:
node_relations_list[-1].head()

,id_etapa4,closest_node
0,1,6156
1,2,1626
2,3,2827
3,4,5319
4,5,13025


### Costos por etapa

Primero la función que calcula el costo entre etapas sucesivas (esto asume que la lista de etapas está ordenada)

In [97]:
def stage_cost(source_table, target_table, cost_column):
    params = {'source': source_table, 'target': target_table, 'cost': cost_column }
    qry_str = """SELECT DISTINCT ON (start_vid)
                 start_vid as id_%(source)s, end_vid as id_%(target)s, agg_cost as costo_%(source)s_%(target)s
          FROM   (SELECT * FROM pgr_dijkstraCost(
              'select id, source, target, %(cost)s as cost from red',
              array(select distinct(s.closest_node) from (select e.*, r.closest_node
                                                        from %(source)s e
                                                        join %(source)s_node r
                                                        on e.id = r.id_%(source)s::int) as s),
              array(select distinct(t.closest_node) from (select e.*, r.closest_node
                                                        from %(target)s e
                                                        join %(target)s_node r
                                                        on e.id = r.id_%(target)s::int) as t),
                 directed:=false)
          ) as sub
          ORDER  BY start_vid, agg_cost asc""" % params
    try:
        df = pd.read_sql(qry_str, engine)
    except ValueError as e:
        print(e)
    return df

Para calcular la distancia entre dos etapas hacemos:

In [92]:
d0 = stage_cost("etapa1", "etapa2", "costo")
d0.head()

,start_etapa1,end_etapa2,costo_etapa1_etapa2
0,412,1851,268660.672500
1,459,1851,223833.071148
2,467,1851,233672.559784
3,638,11230,100905.641581
4,639,11230,105890.189821


Y para calcular todas las distancias:

In [107]:
distancias = []
for i, etapa in enumerate(etapas): 
    if i < len(etapas)-1:
        stage = stage_cost(etapa,etapas[i+1], "costo")
        cost_col = list(stage.columns)[-1]
        stage.columns = ['start_' + str(i), 'end_' + str(i), cost_col]
        distancias.append(stage)

In [108]:
distancias[0].head()
#list(distancias[0])[-1]

,start_0,end_0,costo_etapa1_etapa2
0,412,1851,268660.672500
1,459,1851,223833.071148
2,467,1851,233672.559784
3,638,11230,100905.641581
4,639,11230,105890.189821


In [109]:
distancias[1].head()

,start_1,end_1,costo_etapa2_etapa3
0,1851,1626,14710.325903
1,4365,2827,5756.321023
2,4664,4252,94517.114344
3,5092,4252,13204.104910
4,5419,8350,11464.748789


In [110]:
distancias[2].head()

,start_2,end_2,costo_etapa3_etapa4
0,1506,2827,3669.320367
1,1626,5319,469.458292
2,2289,8350,10649.014814
3,2827,1506,3669.320367
4,4252,1506,53383.568474


In [121]:
c = pd.merge(distancias[0], distancias[1], left_on='end_0', right_on='start_1')
d = pd.merge(c, distancias[2], left_on='end_1', right_on='start_2')
d.head()

,start_0,end_0,costo_etapa1_etapa2,start_1,end_1,costo_etapa2_etapa3,start_2,end_2,costo_etapa3_etapa4
0,412,1851,268660.672500,1851,1626,14710.325903,1626,5319,469.458292
1,459,1851,223833.071148,1851,1626,14710.325903,1626,5319,469.458292
2,467,1851,233672.559784,1851,1626,14710.325903,1626,5319,469.458292
3,714,1851,118419.029160,1851,1626,14710.325903,1626,5319,469.458292
4,808,1851,32954.196776,1851,1626,14710.325903,1626,5319,469.458292


In [125]:
for i, distancia in enumerate(distancias):
    print(i)
    if i == 0:
        costos = pd.merge(distancia, distancias[1], left_on='end_0', right_on='start_1')
        print(list(costos.columns))
        #cost_cols = [c for c in costos.columns if c.startswith('costo')]
        #columnas = ['start_0', 'end_1'] + cost_cols
        #costos = costos[columnas]
    elif i < len(distancias) - 1:
        costos = pd.merge(costos, distancias[i+1], left_on='end_' + str(i), right_on='start_' + str(i+1))
        print(list(costos.columns))

0
['start_0', 'end_0', 'costo_etapa1_etapa2', 'start_1', 'end_1', 'costo_etapa2_etapa3']
1
['start_0', 'end_0', 'costo_etapa1_etapa2', 'start_1', 'end_1', 'costo_etapa2_etapa3', 'start_2', 'end_2', 'costo_etapa3_etapa4']
2


In [126]:
costos.head()

,start_0,end_0,costo_etapa1_etapa2,start_1,end_1,costo_etapa2_etapa3,start_2,end_2,costo_etapa3_etapa4
0,412,1851,268660.672500,1851,1626,14710.325903,1626,5319,469.458292
1,459,1851,223833.071148,1851,1626,14710.325903,1626,5319,469.458292
2,467,1851,233672.559784,1851,1626,14710.325903,1626,5319,469.458292
3,714,1851,118419.029160,1851,1626,14710.325903,1626,5319,469.458292
4,808,1851,32954.196776,1851,1626,14710.325903,1626,5319,469.458292


Ahora hay que unir cada resultado a la geometría que le corresponde, esto es:

- etapa1 -> costo de 1 a 2
- etapa2 -> costo de 2 a 3
- etapa(k-1) -> costo de k-1 a k

Primero leemos los datos originales

In [88]:
etapas_gdfs = []
for etapa in etapas:
    sql = """select * from %(etapa)s""" % {"etapa":etapa}
    etapas_gdfs.append(gpd.GeoDataFrame.from_postgis(sql, con, geom_col='geom'))
#etapas_gdfs[0].head()

In [61]:
etapas_gdfs[3].head()

,id,geom,oid_,name,folderpath,symbolid,altmode,base,snippet,popupinfo,haslabel,labelid,point_x,point_y,point_z,point_m,nodos_exportadora
0,1,POINT (529582.4613746653 1754712.263455871),0,Campesinos Ecológicos de la Sierra Madre de Ch...,exportadoras de cafe.kml/Mis lugares,0,-1,0.0,None,None,-1,0,-92.723684,15.871240,0.0,1.0,6156
1,2,POINT (488997.568839125 1851966.324229623),0,"Expogranos Mexicanos, S.A. de C.V.",exportadoras de cafe.kml/Mis lugares,0,-1,0.0,None,None,-1,0,-93.103229,16.750582,0.0,1.0,1626
2,3,POINT (497919.4919888135 1849143.182136836),0,Federación Indígena Ecológica de Chiapas S.S.S.,exportadoras de cafe.kml/Mis lugares,0,-1,0.0,None,None,-1,0,-93.019517,16.725086,0.0,1.0,2827
3,4,POINT (489271.3499157729 1851867.942677076),0,"Los Corzo de Cerro Brujo, S.C. de R.L. de C.V.",exportadoras de cafe.kml/Mis lugares,0,-1,0.0,None,None,-1,0,-93.100660,16.749694,0.0,1.0,5319
4,5,POINT (608152.1637896972 1803965.131279904),0,"Unión de Ejidos de la Selva, S.C. de R.L. de C.V",exportadoras de cafe.kml/Mis lugares,0,-1,0.0,None,None,-1,0,-91.987600,16.314252,0.0,1.0,13025


Luego, para todas las etapas, menos la última, unimos las geometrías con los costos. Pero primero a través de las relaciones entre nodos y etapas:

In [89]:
etapas_costo = []
for i, (etapa,relacion) in enumerate(zip(etapas, node_relations_list)):
    if i < len(node_relations_list) - 1:
        etapas_costo.append(pd.merge(relacion, distancias[i], 
                                     left_on='closest_node', 
                                     right_on='start_vid', how='right'))    

KeyError: 'start_vid'

In [65]:
etapas_costo[1].head()

,id_etapa2,closest_node,start_vid,end_vid,costo_etapa2_etapa3
0,1,12401,12401,4252,230551.035276
1,2,12536,12536,12401,12846.434991
2,3,23378,23378,12401,5194.924265
3,4,12671,12671,11386,1222.645731
4,5,17305,17305,11386,159388.510842


Creamos una lista de GeoDataFrames con los costos por etapa

In [50]:
gdfs_costo = []
for i, (etapa, costo) in enumerate(zip(etapas[0:-1], etapas_costo)):
    gdfs_costo.append(pd.merge(etapas_gdfs[i], costo, left_on='id', right_on='id_' + etapa, how='right'))

In [67]:
gdfs_costo[2].head()

,id,geom,id_etapa3,closest_node,start_vid,end_vid,costo_etapa3_etapa4
0,1,(POINT (529582.4613746654 1754712.263455767)),1,6156,6156,1506,154742.825662
1,2,(POINT (488997.5688391249 1851966.324229523)),2,1626,1626,5319,469.458292
2,3,(POINT (497919.4919888134 1849143.182136733)),3,2827,2827,1506,3669.320367
3,4,(POINT (489271.3499157728 1851867.942676975)),4,5319,5319,1626,469.458292
4,5,(POINT (608152.1637896977 1803965.131279801)),5,13025,13025,11386,31208.731277


In [68]:
gdfs_costo[1].head()

,id,geom,id_etapa2,closest_node,start_vid,end_vid,costo_etapa2_etapa3
0,1,(POINT (571294.2455646662 1898844.477570399)),1,12401,12401,4252,230551.035276
1,2,(POINT (577131.1944325645 1891906.774007753)),2,12536,12536,12401,12846.434991
2,3,(POINT (571400.0710233421 1897846.608146797)),3,23378,23378,12401,5194.924265
3,4,(POINT (594520.7830304323 1794525.863464239)),4,12671,12671,11386,1222.645731
4,5,(POINT (576303.8689003233 1740333.669975902)),5,17305,17305,11386,159388.510842


In [76]:
pd.merge(gdfs_costo[0],gdfs_costo[1],left_on='end_vid', 
        right_on='start_vid', 
        how='left',
        suffixes=['','_y']).head()

,id_0,geom,id,id_ac,id_etapa1,closest_node,start_vid,end_vid,costo_etapa1_etapa2,id_y,geom_y,id_etapa2,closest_node_y,start_vid_y,end_vid_y,costo_etapa2_etapa3
0,1,POINT (645736.3108251573 1808481.380637101),1737,07052028-2002,1737,16385,16385,12671,88197.560033,4.0,(POINT (594520.7830304323 1794525.863464239)),4.0,12671.0,12671.0,11386.0,1222.645731
1,3,POINT (551019.6914652064 1860672.211684213),2,07093002-0001,2,7228,7228,5092,43716.671817,16.0,(POINT (539067.9358458845 1846810.951025826)),16.0,5092.0,5092.0,4252.0,13204.104910
2,4,POINT (587381.6952086436 1652313.728707811),3,07102002-1010,3,19701,19701,19558,74410.673774,18.0,(POINT (558173.7460236264 1674299.355523742)),18.0,19558.0,19558.0,21474.0,2154.781930
3,6,POINT (475834.8140944368 1880266.266886614),5,07079002-1001,5,22374,22374,1851,85095.821348,13.0,(POINT (479696.1325948501 1854348.190015199)),13.0,1851.0,1851.0,1626.0,14710.325903
4,7,POINT (572919.1574849291 1922516.347539169),6,07077008-4009,6,15217,15217,11230,66259.151528,10.0,(POINT (570611.4481645139 1899880.516466462)),10.0,11230.0,11230.0,12401.0,1972.427177


In [77]:
e12 = pd.merge(e01,gdfs_costo[2],left_on='end_vid_y', 
        right_on='start_vid', 
        how='left',
        suffixes=['','_y'])
e12.head()

,id_0,geom_x,id_x,id_ac,id_etapa1,closest_node_x,start_vid_x,end_vid_x,costo_etapa1_etapa2,id_y,...,start_vid_y,end_vid_y,costo_etapa2_etapa3,id,geom,id_etapa3,closest_node,start_vid,end_vid,costo_etapa3_etapa4
0,1,POINT (645736.3108251573 1808481.380637101),1737,07052028-2002,1737,16385,16385,12671,88197.560033,4.0,...,12671.0,11386.0,1222.645731,17.0,(POINT (593982.6286475633 1793476.678453411)),17.0,11386.0,11386.0,13025.0,31208.731277
1,3,POINT (551019.6914652064 1860672.211684213),2,07093002-0001,2,7228,7228,5092,43716.671817,16.0,...,5092.0,4252.0,13204.104910,7.0,(POINT (536190.4028605289 1851678.290711182)),7.0,4252.0,4252.0,1506.0,53383.568474
2,4,POINT (587381.6952086436 1652313.728707811),3,07102002-1010,3,19701,19701,19558,74410.673774,18.0,...,19558.0,21474.0,2154.781930,19.0,(POINT (557182.4880107472 1672991.847419234)),19.0,21474.0,21474.0,17278.0,52449.489463
3,6,POINT (475834.8140944368 1880266.266886614),5,07079002-1001,5,22374,22374,1851,85095.821348,13.0,...,1851.0,1626.0,14710.325903,2.0,(POINT (488997.5688391249 1851966.324229523)),2.0,1626.0,1626.0,5319.0,469.458292
4,7,POINT (572919.1574849291 1922516.347539169),6,07077008-4009,6,15217,15217,11230,66259.151528,10.0,...,11230.0,12401.0,1972.427177,16.0,(POINT (571291.216900418 1898845.667823051)),16.0,12401.0,12401.0,4252.0,230551.035276


Ahora creamos un nuevo geoDataframe con valores de los costos por etapa y total

In [56]:
costos_totales = reduce(lambda x, y,: pd.merge(x, y, left_on='end_vid', 
                                               right_on='start_vid', 
                                               how='left',
                                               suffixes=['','_y']), gdfs_costo)
#costos_totales = reduce(lambda x, y: pd.merge(x, y, left_on='start_vid', right_on='end_vid', how='right'), gdfs_costo)
#costos_totales["cost_tot"] = costos_totales.iloc[:, costos_totales.columns.str.contains('costo_')].sum(1)
pd.merge(gdfs_costo[0],gdfs_costo[1],left_on='end_vid', 
        right_on='start_vid', 
        how='left',
        suffixes=['','_y']).head()

,id_0,geom,id,id_ac,id_etapa1,closest_node,start_vid,end_vid,costo_etapa1_etapa2,id_y,...,start_vid_y,end_vid_y,costo_etapa2_etapa3,id_y,geom_y,id_etapa3,closest_node_y,start_vid_y,end_vid_y,costo_etapa3_etapa4
0,1,POINT (645736.3108251573 1808481.380637101),1737,07052028-2002,1737,16385,16385,12671,88197.560033,4.0,...,12671.0,11386.0,1222.645731,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,POINT (551019.6914652064 1860672.211684213),2,07093002-0001,2,7228,7228,5092,43716.671817,16.0,...,5092.0,4252.0,13204.104910,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,POINT (587381.6952086436 1652313.728707811),3,07102002-1010,3,19701,19701,19558,74410.673774,18.0,...,19558.0,21474.0,2154.781930,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,POINT (475834.8140944368 1880266.266886614),5,07079002-1001,5,22374,22374,1851,85095.821348,13.0,...,1851.0,1626.0,14710.325903,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,POINT (572919.1574849291 1922516.347539169),6,07077008-4009,6,15217,15217,11230,66259.151528,10.0,...,11230.0,12401.0,1972.427177,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
for i,etapa in enumerate(etapas[:-1]):
    costos_totales["cost_"+etapas[0] + "_" + etapas[i+1]] = \
    costos_totales.iloc[:, costos_totales.columns.str.contains('costo_')].sum(1)
    print("costo_" + etapa + "_" + etapas[i+1])

costo_etapa1_etapa2
costo_etapa2_etapa3
costo_etapa3_etapa4


In [21]:
nuevas_columnas = [c for c in costos_totales.columns if c.startswith("costo")]
nuevas_columnas = ["id_" + etapas[0], "geom"] + nuevas_columnas 
solo_costos = costos_totales[nuevas_columnas]
len(solo_costos)

2647

In [39]:
sumas_acumuladas = solo_costos.iloc[:, solo_costos.columns.str.contains('costo_')].cumsum(axis=1)
sumas_acumuladas = sumas_acumuladas.merge(solo_costos.iloc[:,[0,1]], left_index=True, right_index=True)
sumas_acumuladas.head()

,costo_etapa1_etapa2,costo_etapa2_etapa3,costo_etapa3_etapa4,id_etapa1,geom
0,88197.560033,89420.205764,NaN,1737,POINT (645736.3108251573 1808481.380637101)
1,43716.671817,56920.776727,NaN,2,POINT (551019.6914652064 1860672.211684213)
2,74410.673774,76565.455704,NaN,3,POINT (587381.6952086436 1652313.728707811)
3,85095.821348,99806.147252,NaN,5,POINT (475834.8140944368 1880266.266886614)
4,66259.151528,68231.578705,NaN,6,POINT (572919.1574849291 1922516.347539169)


In [41]:
sumas_acumuladas = gpd.GeoDataFrame(sumas_acumuladas, geometry=sumas_acumuladas.geom)
sumas_acumuladas.drop(['geom'], axis=1, inplace=True)
sumas_acumuladas.to_file(driver= 'ESRI Shapefile', filename= "idw/result.shp")

AttributeError: 'GeoDataFrame' object has no attribute 'geom'

In [33]:
sumas_acumuladas.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2647 entries, 0 to 2646
Data columns (total 5 columns):
costo_etapa1_etapa2    2647 non-null float64
costo_etapa2_etapa3    2645 non-null float64
costo_etapa3_etapa4    11 non-null float64
id_etapa1              2647 non-null int64
geom                   2647 non-null object
dtypes: float64(3), int64(1), object(1)
memory usage: 204.1+ KB


In [45]:
for columna in sumas_acumuladas:
    if columna.startswith('costo_'):
            comando = ['gdal_grid', '-zfield', 'costo_tota', '-l', 'costo', '-a',
               'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0',
               '-of', 'GTiff', '/home/eurekastein/Documentos/cadenas.py/idw/costo'+ columna+'.shp', 
               '/home/eurekastein/Documentos/cadenas.py/idw/costo'+ columna+'.tif']
    try:
        call(comando)
        print(comando)
    except:
        print('valió verga')
    print('La columna '+columna+' no valió verga')
   

['gdal_grid', '-zfield', 'costo_tota', '-l', 'costo', '-a', 'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0', '-of', 'GTiff', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa1_etapa2.shp', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa1_etapa2.tif']
La columna costo_etapa1_etapa2 no valió verga
['gdal_grid', '-zfield', 'costo_tota', '-l', 'costo', '-a', 'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0', '-of', 'GTiff', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa2_etapa3.shp', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa2_etapa3.tif']
La columna costo_etapa2_etapa3 no valió verga
['gdal_grid', '-zfield', 'costo_tota', '-l', 'costo', '-a', 'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0', '-of', 'GTiff', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa

Nuevo intento de IDW

In [31]:
def readPoints(dataFile, Zfield='Z'):
    data = {}
    xv=[]
    yv=[]
    values=[]
    ds = ogr.Open(dataFile)
    if ds is None:
       raise Exception('Could not open ' + dataFile)
    
    layer = ds.GetLayer()
    proj = layer.GetSpatialRef()
    extent = layer.GetExtent()

    feature = layer.GetNextFeature()
    if feature.GetFieldIndex(zField) == -1:
         raise Exception('zField is not valid: ' + zField)

    while feature:
        geometry = feature.GetGeometryRef()
        xv.append(geometry.GetX())
        yv.append(geometry.GetY())
        values.append(feature.GetField(zField))

        feature = layer.GetNextFeature()
    data['extent'] = extent 
    data['xv']=xv
    data['yv']=yv
    data['values']=values
    data['proj'] = proj
    ds = None
    return data